# NOAA extreme weather events
The [National Oceanic and Atmospheric Administration](https://en.wikipedia.org/wiki/National_Oceanic_and_Atmospheric_Administration) has a database of extreme weather events that contains lots of detail for every year. [Link](https://www.climate.gov/maps-data/dataset/severe-storms-and-extreme-events-data-table).  I'll extract the data for 2018.

In [218]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)  # Unlimited columns

## Get official list of US counties

In [219]:
import geopandas
# Import a shape file with all the counties in the US.
counties = geopandas.read_file('../data_input/1_USCounties/')

# Turn state codes from strings to integers
for col in ['STATE_FIPS', 'CNTY_FIPS', 'FIPS']:
    counties[col] = counties[col].astype(int)

# Set the FIPS code as the index, and sort by it
counties = counties.set_index('FIPS').sort_index(axis=0)
    
# Make a list of the valid FIPS codes
official_counties = counties.index.tolist()

# Samples
random.sample(official_counties, 5)

[18141, 40041, 26081, 6033, 56039]

In [234]:
## From the official list of counties, get a dictionary of FIPS state codes

# Extract names and codes
temp1 = counties[['STATE_NAME', 'STATE_FIPS']].copy()
# Turn into dictionary
state_FIPS_codes = temp1.groupby('STATE_NAME').max().to_dict()['STATE_FIPS']

# Samples
random.sample(list(state_FIPS_codes.items()), 5)

[('Wyoming', 56),
 ('Florida', 12),
 ('New Mexico', 35),
 ('Maine', 23),
 ('Arizona', 4)]

## Make dictionary of state abbreviations

In [235]:
# Add the full state name to the dafatframe
state_codes = pd.read_csv('../data_input/states_code.csv')
state_codes['state_allcaps'] = state_codes['State'].str.upper()
state_codes = state_codes.set_index('state_allcaps')
state_codes = state_codes.to_dict()['Abbreviation']

# Samples
random.sample(list(state_codes.items()), 5)

[('GEORGIA', 'GA'),
 ('MISSOURI', 'MO'),
 ('MISSISSIPPI', 'MS'),
 ('DELAWARE', 'DE'),
 ('MONTANA', 'MT')]

## Make dictionary of weather zones
Many of the FIPS codes in the NOAA dataset correspond not to counties, but to National Weather Service (NWS) Forecast Zones.  Some zones contain several counties, so we want to make sure that each zone-level weather event gets recorded in all the corresponding counties. NWS publishes the zone-county correlations [here](https://www.weather.gov/gis/ZoneCounty).

In [173]:
# Get zone-county correlations
url = 'https://www.weather.gov/source/gis/Shapefiles/County/bp02ap19.dbx'
cols = ['STATE','ZONE','CWA','NAME','STATE_ZONE','COUNTY','FIPS',
        'TIME_ZONE','FE_AREA','LAT','LON']
zones = pd.read_csv(url, delimiter='|', names=cols).sort_values('FIPS')

The column `FIPS` refers to the county FIPS.

In [39]:
print(zones.shape)
zones.head()

(4536, 11)


,STATE,ZONE,CWA,NAME,STATE_ZONE,COUNTY,FIPS,TIME_ZONE,FE_AREA,LAT,LON
165,AL,41,BMX,Autauga,AL041,Autauga,1001,C,c,32.5349,-86.6428
195,AL,266,MOB,Baldwin Coastal,AL266,Baldwin,1003,C,sw,30.4940,-87.6699
191,AL,262,MOB,Baldwin Inland,AL262,Baldwin,1003,C,sw,30.4940,-87.6699
193,AL,264,MOB,Baldwin Central,AL264,Baldwin,1003,C,sw,30.9471,-87.7615
174,AL,50,BMX,Barbour,AL050,Barbour,1005,C,se,31.8696,-85.3932


This file has a different naming convention for the state zone than what I used. I'll have to make some conversions in order to integrate the two lists.

In [95]:


# # Add the state FIPS codes for the states in our official list
# # (produces NANs for non-state territories)
# zones['state_FIPS'] = zones['state'].map(state_FIPS_codes)

# # Drop rows for non-state territories
# zones = zones.dropna(subset=['state_FIPS'])

# # Turn to ints and then strings, since the appearance of NANs 
# # had cast this column as floats.
# zones['state_FIPS'] = zones['state_FIPS'].astype(int).astype(str)

# # Frem each STATE_ZONE, extract the three characters corresponding 
# # to the county FIPS code.
# zones['state_zone'] = [x[2:] for x in zones['STATE_ZONE']]

# # Create a new FIPS code for the zone, which uses the state numeric code.
# zones['zone_FIPS'] = (zones['state_FIPS'] + zones['state_zone']).astype(int)

# # Cast the two FIPS columns that we care about as ints
# zones['FIPS'] = zones['FIPS'].astype(int)
# zones['zone_FIPS'] = zones['zone_FIPS'].astype(int)

# zones.head()

,STATE,ZONE,CWA,NAME,STATE_ZONE,COUNTY,FIPS,TIME_ZONE,FE_AREA,LAT,LON,state,state_FIPS,state_zone,zone_FIPS
165,AL,41,BMX,Autauga,AL041,Autauga,1001,C,c,32.5349,-86.6428,Alabama,1,041,1041
195,AL,266,MOB,Baldwin Coastal,AL266,Baldwin,1003,C,sw,30.4940,-87.6699,Alabama,1,266,1266
191,AL,262,MOB,Baldwin Inland,AL262,Baldwin,1003,C,sw,30.4940,-87.6699,Alabama,1,262,1262
193,AL,264,MOB,Baldwin Central,AL264,Baldwin,1003,C,sw,30.9471,-87.7615,Alabama,1,264,1264
174,AL,50,BMX,Barbour,AL050,Barbour,1005,C,se,31.8696,-85.3932,Alabama,1,050,1050


In [238]:
# Create a dictionary with NWS zones as keys, each 
# corresponding to one or more county FIPS codes.
zone_dict = {}
for row in zones[['FIPS','STATE_ZONE']].iterrows():
    county = row[1]['FIPS']
    zone = row[1]['STATE_ZONE']
    
    if zone not in zone_dict:
        zone_dict[zone] = [county]
    else:
        zone_dict[zone].append(county)
        
# Samples
random.sample(list(zone_dict.items()), 5)

[('CA055', [6071]),
 ('TN028', [47189]),
 ('MT007', [30023, 30039, 30077, 30093]),
 ('NE054', [31105]),
 ('CO047', [8039, 8073])]

## Exploring one year of NOAA data
NOAA data comes year-by-year.  Here is an exploration of that one year, which will allow me to make a function that does this for any year.

Note that the field `CZ_FIPS` can correspond to either a county FIPS or a NWS zone, and some numbers overlap (for example, `001` in Alabama can correspond to either Autauga county or the Lauderdale NWS zone; see below).  Therefore, I will process county- and zone-level events separately.  The goal is to match each event to all the counties where it happens, and make sure that those counties are identified with the same 4-5 digit FIPS code as our canonical list of official counties.  That number has 1-2 digits that correspond to the state, and 3 digits that correspond to the county (e.g., .

In [169]:
df1[df1['STATE'] == 'ALABAMA'][df1['CZ_FIPS'] == 1].loc[13273:14072][['STATE','EVENT_TYPE','CZ_TYPE','CZ_FIPS','CZ_NAME']]

/anaconda3/envs/sand2/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,STATE,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME
13273,ALABAMA,Tornado,C,1,AUTAUGA
14072,ALABAMA,Tropical Depression,Z,1,LAUDERDALE


### Initial processing

In [258]:
# The full table contains more than I want to use.
df1 = pd.read_csv('../data_local/NOAA/StormEvents_details-ftp_v1.0_d2018_c20190422.csv')
print(df1.shape)
print(df1.columns)
df1.head(2)

(62169, 51)
Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,CZ_TYPE,CZ_FIPS,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,FLOOD_CAUSE,CATEGORY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201806,6,1810,201806,6,1810,125578,753161,NEBRASKA,31,2018,June,Hail,C,69,GARDEN,LBF,06-JUN-18 18:10:00,MST-7,06-JUN-18 18:10:00,0,0,0,0,0.00K,0.00K,Public,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,N,OSHKOSH,36.0,N,OSHKOSH,41.93,-102.21,41.93,-102.21,Severe storms developed in the Nebraska Panhan...,Hail predominately penny size with some quarte...,CSV
1,201806,6,1741,201806,6,1741,125578,753160,NEBRASKA,31,2018,June,Hail,C,161,SHERIDAN,LBF,06-JUN-18 17:41:00,MST-7,06-JUN-18 17:41:00,0,0,0,0,0.00K,0.00K,Trained Spotter,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NW,BINGHAM,1.0,NW,BINGHAM,42.03,-102.10,42.03,-102.10,Severe storms developed in the Nebraska Panhan...,Hail mainly quarter size with some half dollar...,CSV


In [267]:
# Extract only a few useful columns
df2 = df1[['STATE','STATE_FIPS','CZ_TYPE','CZ_FIPS','EVENT_TYPE']].copy()

# Add a column with the two-letter state code
df2['state'] = df2['STATE'].map(state_codes)

# Remove any entries that don't happen in states on the official list
# (this excludes entries from lakes, oceans, and territories)
df2 = df2[df2['STATE_FIPS'].isin(state_FIPS_codes.values())]

print(df2.shape)
df2.head(2)

(59669, 6)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state
0,NEBRASKA,31,C,69,Hail,NE
1,NEBRASKA,31,C,161,Hail,NE


In [268]:
# Split the dataframe into county- and zone-level events
df2_counties = df2[df2.CZ_TYPE == 'C'].copy()
df2_zones = df2[df2.CZ_TYPE == 'Z'].copy()

df2_counties.shape, df2_zones.shape

((35561, 6), (24108, 6))

In [363]:
# In order to aggregate the county- and zone-level events, I'll create
# a single dictionary of official zipcodes and event types.

zone_events = {}
for fips in official_counties:
    
event_types

# These are all event types from the metadata, edited to remove marine events
event_types = ['Astronomical Low Tide','Avalanche','Blizzard','Coastal Flood',
               'Cold/Wind Chill','Debris Flow','Dense Fog','Dense Smoke',
               'Drought','Dust Devil','Dust Storm','Excessive Heat',
               'Extreme Cold/Wind Chill','Flash Flood','Flood','Frost/Freeze',
               'Funnel Cloud','Freezing Fog','Hail','Heat','Heavy Rain',
               'Heavy Snow','High Surf','High Wind','Hurricane (Typhoon)',
               'Ice Storm','Lake-Effect Snow','Lakeshore Flood','Lightning',
               'Rip Current','Seiche','Sleet','Storm Surge/Tide','Strong Wind',
               'Thunderstorm Wind','Tornado','Tropical Depression',
               'Tropical Storm','Tsunami','Volcanic Ash','Wildfire',
               'Winter Storm','Winter Weather',]

### Process county-level events
By combining the state FIPS and the county FIPS and casting as int, making sure that the county FIPS is 3 digits.

In [298]:
print(df2_counties.shape)
df2_counties.head(2)

(35518, 7)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state,FIPS
0,NEBRASKA,31,C,069,Hail,NE,31069
1,NEBRASKA,31,C,161,Hail,NE,31161


In [287]:
# Re-cast FIPS codes as strings
df2_counties['STATE_FIPS'] = df2_counties['STATE_FIPS'].astype(str)
df2_counties['CZ_FIPS'] = df2_counties['CZ_FIPS'].astype(str)

# Make sure the string for CZ_FIPS is 3 digits long.
FIPS_2 = []
for fip in df2_counties['CZ_FIPS']:
    if len(fip) == 3:
        fip2 = fip
    elif len(fip) == 2:
        fip2 = '0' + fip
    elif len(fip) == 1:
        fip2 = '00' + fip
    FIPS_2.append(fip2)

df2_counties['CZ_FIPS'] = FIPS_2

# Create a full FIPS for each county, cast as int.
complete_FIPS = [int(x+y) for x, y in zip(df2_counties['STATE_FIPS'],
                                          df2_counties['CZ_FIPS'])]
df2_counties['FIPS'] = complete_FIPS

# Get rid of any events for counties not on the official list
df2_counties = df2_counties[df2_counties.FIPS.isin(official_counties)]

print(df2_counties.shape)
df2_counties.head()

(35518, 7)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state,FIPS
0,NEBRASKA,31,C,069,Hail,NE,31069
1,NEBRASKA,31,C,161,Hail,NE,31161
2,VERMONT,50,C,011,Thunderstorm Wind,VT,50011
3,VERMONT,50,C,015,Thunderstorm Wind,VT,50015
4,NEBRASKA,31,C,135,Tornado,NE,31135


### Process zone-level events
By making sure that the zone code is 3 digits, combining it with the state 2-letter code, and using the dictionary of zone codes to expand each zone-level event into as many county-level events as it corresponds to.

In [289]:
print(df2_zones.shape)
df2_zones.head(2)

(24108, 6)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state
41,TEXAS,48,Z,158,Drought,TX
42,OKLAHOMA,40,Z,14,Wildfire,OK


In [353]:
# Re-cast FIPS codes as strings
df2_zones['CZ_FIPS'] = df2_zones['CZ_FIPS'].astype(str)

# Make sure the string for CZ_FIPS is 3 digits long.
FIPS_2 = []
for fip in df2_zones['CZ_FIPS']:
    if len(fip) == 3:
        fip2 = fip
    elif len(fip) == 2:
        fip2 = '0' + fip
    elif len(fip) == 1:
        fip2 = '00' + fip
    FIPS_2.append(fip2)

df2_zones['CZ_FIPS'] = FIPS_2

# Add the two-letter state code
df2_zones['Zone_FIPS'] = df2_zones['state'] + df2_zones['CZ_FIPS']

# Add the full list of county FIPS codes
df2_zones['FIPS'] = df2_zones['Zone_FIPS'].map(zone_dict)

# Drop any rows with NANs in FIPS.
df2_zones = df2_zones.dropna(subset=['FIPS'])

print(df2_zones.shape)
df2_zones.head(2)

(23692, 8)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state,FIPS,Zone_FIPS
41,TEXAS,48,Z,158,Drought,TX,[48027],TX158
42,OKLAHOMA,40,Z,014,Wildfire,OK,[40129],OK014


In [362]:


short = df2_zones.head(20).copy()

for row in short.iterrows():
    # list of FIPS code for this zone
    fips = row[1]['FIPS']

    # For every code in the list
    for fip in fips:
        if fip not in zone_events:
            zone_events[fip] == 

        
print(df2_z2.shape)
df2_z2.head(2)

48027
40129
40151
54041
54017
54091
35027
35053
35057
35061
48175
48479
29221
29099
17013
17083
21109
48283
29055
29071
29113
29183
29189
(23, 8)


,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state,FIPS,Zone_FIPS
0,TEXAS,48,Z,158,Drought,TX,48027,TX158
1,OKLAHOMA,40,Z,014,Wildfire,OK,40129,OK014


In [350]:
df2_zones[df2_zones['FIPS'].isnull()]

,STATE,STATE_FIPS,CZ_TYPE,CZ_FIPS,EVENT_TYPE,state,FIPS,Zone_FIPS
98,CALIFORNIA,6,Z,095,High Wind,CA,NaN,CA095
105,CALIFORNIA,6,Z,095,High Wind,CA,NaN,CA095
114,CALIFORNIA,6,Z,098,High Wind,CA,NaN,CA098
706,CALIFORNIA,6,Z,092,Dense Fog,CA,NaN,CA092
708,NORTH CAROLINA,37,Z,103,Winter Storm,NC,NaN,NC103
711,CALIFORNIA,6,Z,090,Dense Fog,CA,NaN,CA090
712,CALIFORNIA,6,Z,091,Dense Fog,CA,NaN,CA091
713,CALIFORNIA,6,Z,097,High Wind,CA,NaN,CA097
750,CALIFORNIA,6,Z,097,High Wind,CA,NaN,CA097
764,CALIFORNIA,6,Z,092,Strong Wind,CA,NaN,CA092


In [335]:
test = df2_zones.loc[41].copy()
test['FIPS'] = 5
test

STATE           TEXAS
STATE_FIPS         48
CZ_TYPE             Z
CZ_FIPS           158
EVENT_TYPE    Drought
state              TX
FIPS                5
Zone_FIPS       TX158
Name: 41, dtype: object

In [ ]:
# # Get rid of any events for counties not on the official list
# # df2_zones = 
# df2_zones[df2_zones.FIPS.isin(official_counties)]

### Keep going

In [ ]:
    
# Create a full FIPS for each county
complete_FIPS = [int(x+y) for x, y in zip(state_FIPS, county_FIPS_2)]
df2['FIPS'] = complete_FIPS


# Drop entries for areas not in the list of counties that we're working
# with.  This includes counties in non-state territories (eg, Puerto Rico),
# areas over lakes and oceans, and several forecast zones that don't 
# correspond to particular counties.
df3 = df2[df2.FIPS.isin(official_counties)].copy()

# Drop obsolete columns
df3 = df3.drop(columns=['STATE_FIPS','CZ_FIPS'])

In [4]:
# Extract only a few useful columns
df2 = df1[['STATE','STATE_FIPS','CZ_TYPE','CZ_FIPS','EVENT_TYPE']].copy()

# Create new column for complete county FIPS code
state_FIPS = [str(x) for x in df2['STATE_FIPS']]
county_FIPS = [str(x) for x in df2['CZ_FIPS']]

# Make sure the string for county FIPS is 3 digits long.
county_FIPS_2 = []
for fip in county_FIPS:
    if len(fip) == 3:
        fip2 = fip
    elif len(fip) == 2:
        fip2 = '0' + fip
    elif len(fip) == 1:
        fip2 = '00' + fip
    county_FIPS_2.append(fip2)
    
# Create a full FIPS for each county
complete_FIPS = [int(x+y) for x, y in zip(state_FIPS, county_FIPS_2)]
df2['FIPS'] = complete_FIPS


# Drop entries for areas not in the list of counties that we're working
# with.  This includes counties in non-state territories (eg, Puerto Rico),
# areas over lakes and oceans, and several forecast zones that don't 
# correspond to particular counties.
df3 = df2[df2.FIPS.isin(official_counties)].copy()

# Drop obsolete columns
df3 = df3.drop(columns=['STATE_FIPS','CZ_FIPS'])

In [5]:
print(df3.shape)
df3.head()

(45865, 3)


,STATE,EVENT_TYPE,FIPS
0,NEBRASKA,Hail,31069
1,NEBRASKA,Hail,31161
2,VERMONT,Thunderstorm Wind,50011
3,VERMONT,Thunderstorm Wind,50015
4,NEBRASKA,Tornado,31135


In [7]:
df4 = df3.copy()

# Create a new column for every event type. Each row has a 1 in the column of
# its event type and a 0 for all others.
for typ in event_types:
    df4[typ] = (typ == df4['EVENT_TYPE']).astype(int)

In [8]:
df4.head()

,STATE,EVENT_TYPE,FIPS,Astronomical Low Tide,Avalanche,Blizzard,Coastal Flood,Cold/Wind Chill,Debris Flow,Dense Fog,Dense Smoke,Drought,Dust Devil,Dust Storm,Excessive Heat,Extreme Cold/Wind Chill,Flash Flood,Flood,Frost/Freeze,Funnel Cloud,Freezing Fog,Hail,Heat,Heavy Rain,Heavy Snow,High Surf,High Wind,Hurricane (Typhoon),Ice Storm,Lake-Effect Snow,Lakeshore Flood,Lightning,Rip Current,Seiche,Sleet,Storm Surge/Tide,Strong Wind,Thunderstorm Wind,Tornado,Tropical Depression,Tropical Storm,Tsunami,Volcanic Ash,Wildfire,Winter Storm,Winter Weather
0,NEBRASKA,Hail,31069,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NEBRASKA,Hail,31161,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,VERMONT,Thunderstorm Wind,50011,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,VERMONT,Thunderstorm Wind,50015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,NEBRASKA,Tornado,31135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [9]:
# Group by the FIPS code, summing up all the events for that county.
# Now the dataset has FIPS code as the index and contains one column for each
# event type.  Each cell is the number of events of that type for that county.
df5 = df4.groupby('FIPS').sum()

In [10]:
print(df5.shape)
df5.head()

(3019, 43)


,Astronomical Low Tide,Avalanche,Blizzard,Coastal Flood,Cold/Wind Chill,Debris Flow,Dense Fog,Dense Smoke,Drought,Dust Devil,Dust Storm,Excessive Heat,Extreme Cold/Wind Chill,Flash Flood,Flood,Frost/Freeze,Funnel Cloud,Freezing Fog,Hail,Heat,Heavy Rain,Heavy Snow,High Surf,High Wind,Hurricane (Typhoon),Ice Storm,Lake-Effect Snow,Lakeshore Flood,Lightning,Rip Current,Seiche,Sleet,Storm Surge/Tide,Strong Wind,Thunderstorm Wind,Tornado,Tropical Depression,Tropical Storm,Tsunami,Volcanic Ash,Wildfire,Winter Storm,Winter Weather
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,6,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,2,1,0,0,0,0,0,1
1003,0,0,0,0,3,0,0,0,0,0,0,0,0,4,1,7,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,1,0,0,0,0,0,1
1005,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,6,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
1007,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
1009,0,0,0,0,3,0,0,0,0,0,0,0,1,0,0,8,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2,0,0,0,0,0,0,3


In [ ]:
# I reindex the dataframe so that it contains a row for every county in
# the official county list. Counties without events are filled with 0.
df6 = df5.reindex(official_counties, fill_value=0)

In [ ]:
print(df6.shape)
df6.head()

In [ ]:
# These are the county-level extreme weather events reported un 2018.
df6.sum().sort_values(ascending=False)

## Processing all NOAA files

In [ ]:
# These are all event types from the metadata, edited to remove marine events
event_types = ['Astronomical Low Tide','Avalanche','Blizzard','Coastal Flood',
               'Cold/Wind Chill','Debris Flow','Dense Fog','Dense Smoke',
               'Drought','Dust Devil','Dust Storm','Excessive Heat',
               'Extreme Cold/Wind Chill','Flash Flood','Flood','Frost/Freeze',
               'Funnel Cloud','Freezing Fog','Hail','Heat','Heavy Rain',
               'Heavy Snow','High Surf','High Wind','Hurricane (Typhoon)',
               'Ice Storm','Lake-Effect Snow','Lakeshore Flood','Lightning',
               'Rip Current','Seiche','Sleet','Storm Surge/Tide','Strong Wind',
               'Thunderstorm Wind','Tornado','Tropical Depression',
               'Tropical Storm','Tsunami','Volcanic Ash','Wildfire',
               'Winter Storm','Winter Weather',]

In [ ]:
def process_noaa(filepath):
    """
    Process one year of NOAA Extreme weather events. Requires
    the list of official counties and the list of official weather
    event types.
    
    Inputs
    ------
    filepath (string) : file path for the CSV data file.
    
    
    Outputs
    -------
    df6 (pandas.DataFrame) : Dataframe with the list of official FIPS
                                    codes as index and one column for each 
                                    disaster type handled by NOAA.
    
    """
    
    df1 = pd.read_csv(filepath)
    
    # Extract only a few important columns
    df2 = df1[['STATE_FIPS','CZ_FIPS','EVENT_TYPE']].copy()
    
    # Remove any rows with nulls
    df2 = df2.dropna()

    # Create new column for complete county FIPS code.
    # Ensure that FIPS are ints, otherwise string manipulation fails
    state_FIPS = [str(x) for x in df2['STATE_FIPS'].astype(int)]
    county_FIPS = [str(x) for x in df2['CZ_FIPS'].astype(int)]


    # Make sure the string for county FIPS is 3 digits long,
    # with trailing zeroes on the left.
    county_FIPS_2 = []
    for fip in county_FIPS:
        if len(fip) == 3:
            fip2 = fip
        elif len(fip) == 2:
            fip2 = '0' + fip
        elif len(fip) == 1:
            fip2 = '00' + fip
        county_FIPS_2.append(fip2)

    # Create a full FIPS for each county
    complete_FIPS = [int(x+y) for x, y in zip(state_FIPS, county_FIPS_2)]
    df2['FIPS'] = complete_FIPS

    # Drop entries for areas not in the list of counties that we're working
    # with.  This includes counties in non-state territories (eg, Puerto Rico),
    # areas over lakes and oceans, and several forecast zones that don't 
    # correspond to particular counties.
    df3 = df2[df2.FIPS.isin(official_counties)].copy()

    # Drop obsolete columns
    df4 = df3.drop(columns=['STATE_FIPS','CZ_FIPS'])

    # Create a new column for every event type. Each row has a 1 in the column of
    # its event type and a 0 for all others.
    for typ in event_types:
        df4[typ] = (typ == df4['EVENT_TYPE']).astype(int)
        
    # Group by the FIPS code, summing up all the events for that county.
    # Now the dataset has FIPS code as the index and contains one column for each
    # event type.  Each cell is the number of events of that type for that county.
    df5 = df4.groupby('FIPS').sum()
    
    # I reindex the dataframe so that it contains a row for every county in
    # the official county list. Counties without events are filled with 0.
    df6 = df5.reindex(official_counties, fill_value=0)
    
    return df6

In [ ]:
import glob
import os

# Read the CSV files for each year going back to 1996 (the first year 
# when many of these event types started being recorded)
path = '../data_local/NOAA/'
filenames = sorted(glob.glob(os.path.join(path, '*.csv')))
years = []
layers = []

# Aggregate the dataframes in a list
for name in filenames:
    year = int(name[49:53])
    print(f'Processing {year}')
    years.append(year)
    layers.append(process_noaa(name))

# Concatenate all these dataframes into a single multi-layer dataframe
noaa = pd.concat(layers, keys=years)

In [ ]:
# Create an ordered list of the event types with highest counts
columns_by_most_events = noaa.groupby(level=0).sum().sum().sort_values(ascending=False).index

# Reorder the columns in that order
noaa = noaa[columns_by_most_events].copy()

# Create new df for comparison
noaa2 = noaa.copy()

# Rename some categories
new_names = {'Heat':'Heat old',
             'Flood':'Flood old', 
             'Winter Weather':'Winter Weather old',
             'Hurricane (Typhoon)':'Hurricane'}

noaa2 = noaa2.rename(columns=new_names)

# Create aggregate categories
noaa2['Winter Weather'] = noaa2['Winter Weather old'] +\
                            noaa2['Winter Storm'] +\
                            noaa2['Heavy Snow'] +\
                            noaa2['Frost/Freeze'] +\
                            noaa2['Freezing Fog'] +\
                            noaa2['Ice Storm'] +\
                            noaa2['Sleet'] +\
                            noaa2['Lake-Effect Snow'] +\
                            noaa2['Cold/Wind Chill'] +\
                            noaa2['Extreme Cold/Wind Chill'] +\
                            noaa2['Blizzard']

noaa2['Flood'] = noaa2['Flood old'] +\
                  noaa2['Flash Flood']+\
                  noaa2['Coastal Flood']+\
                  noaa2['Storm Surge/Tide']+\
                  noaa2['Lakeshore Flood']+\
                  noaa2['Debris Flow']               
                                
noaa2['Storm'] = noaa2['Thunderstorm Wind']+\
                  noaa2['High Wind']+\
                  noaa2['Funnel Cloud']+\
                  noaa2['Dust Storm']+\
                  noaa2['Strong Wind']+\
                  noaa2['Dust Devil']+\
                  noaa2['Tropical Depression']+\
                  noaa2['Lightning']+\
                  noaa2['Tropical Storm']+\
                  noaa2['High Surf']+\
                  noaa2['Heavy Rain']+\
                  noaa2['Hail']
    
noaa2['Fire'] = noaa2['Wildfire'] +\
                  noaa2['Dense Smoke']
    
noaa2['Heat'] = noaa2['Heat old'] +\
                  noaa2['Excessive Heat']

final_columns = ['Storm', 'Flood', 'Winter Weather', 'Tornado', 
                 'Drought', 'Heat', 'Fire', 'Hurricane']

# Some categories are excluded because they're not important enough
deliberately_ignored = ['Dense Fog', 'Rip Current','Astronomical Low Tide','Avalanche',
       'Seiche', 'Tsunami', 'Volcanic Ash']

noaa3 = noaa2[final_columns]

In [ ]:
# Aggregate data by county (years summed up)
noaa_county = noaa3.groupby(level=1).sum()

# Aggregate data by year (counties summed up)
noaa_year = noaa3.groupby(level=0).sum()

# Put the data by county into the geopandas file with county shapes
noaa_county_map = counties.merge(noaa_county, on='FIPS', how='left')

In [ ]:
noaa_county_log = np.log(noaa_county+1)
noaa_county_map_log = counties.merge(noaa_county_log, on='FIPS', how='left')
noaa_county_map_log.head()

In [ ]:
print(noaa_county_map.shape)
noaa_county_map.head()

In [ ]:
noaa_county_map_log.plot(column='Storm', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Flood', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Winter Weather', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Tornado', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Drought', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Heat', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Fire', figsize=(20,6), legend=True);

In [ ]:
noaa_county_map_log.plot(column='Hurricane', figsize=(20,6), legend=True);

In [372]:
random.choice(['brian', 'kim','arthur','laryna'])
# ['shreyas','manjula','dma' ]

'kim'

In [371]:
random.choice(['shreyas','manjula','dma' ])

'shreyas'

In [ ]:
random.choice(['brian', 'kim', 'taylor','arthur',
               'laryna','shreyas','manjula','dma'])